In [1]:
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import scipy

In [2]:
with open('./ASTR19_S22_group_project_data.txt', 'r') as file:
    data = file.read()

In [3]:
def parse_tide_data(data):
    rows = []
    for line in data.split('\n'):
        parts = line.split()
        if len(parts) == 3:
            day, time, height = parts
            rows.append({'Day': int(day), 'Time': time, 'Tide Height (feet)': float(height)})
    return pd.DataFrame(rows)

In [4]:
df = parse_tide_data(data)
print(df['Time'])

0      8:07
1     15:37
2      8:55
3     16:25
4      9:45
      ...  
77    11:15
78     4:29
79    12:22
80     5:21
81    13:14
Name: Time, Length: 82, dtype: object


In [5]:
from scipy.optimize import curve_fit

In [ ]:
def oscillatory_function(x, a, b, c, d):
    return a * np.sin(b * x + c) + d
